## Exercise 1

Åbn 'https://www.dba.dk/biler/'.
Find ind alle biler med mærker: 'Peugeot', 'Volvo' og gem dem.

In [101]:
from tqdm import tqdm
import bs4
import requests
import re
    
def scrape_for_cars(url, search_year_flag, year_to_filter_by):
    """
    returns:
        A list of tuples of strings with information about cars of the type Peugot
    """
    
    r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    r.raise_for_status()

    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    event_cells = soup.select("tr[class^=dbaListing]")
    
    scraped_cars_per_page = []
    
    print(len(event_cells))
    
    for event_cell in event_cells:
        
        to_find = re.compile(r'title="Modelår">[\r\n]+([^\r\n]+)')
        regex_car_year = to_find.search(str(event_cell))
        
        car_year = str(event_cell.select("td[title^=Modelår]")[0].getText()).strip()
        car_km = event_cell.select("td[title^=Km]")[0].getText().strip()
        car_km = car_km.replace(".", "")
        car_price = event_cell.select("td[title^=Pris]")[0].getText().strip()     
        car_price = car_price[:-4]
        car_price = car_price.replace(".","")
        car_description = event_cell.select("a[class^=listingLink]")[0].getText().strip()
        #print("Description:")
        #print(car_description
        
        for link in event_cell.find_all('a'):
            if not link.get('href').startswith('https'):
                continue
            car_link = link.get('href')
        
        if(search_year_flag):       
            if(str(regex_car_year.group(1)).strip() == str(year_to_filter_by)):
                scraped_cars_per_page.append((car_link, car_year, car_km, car_price))
        else:
            scraped_cars_per_page.append((car_link, car_year, car_km, car_price))
    
    print(scraped_cars_per_page)
              
    return scraped_cars_per_page


scraped_total_cars = []

indexes = range(1, 6)
search_year_flag = True
year_to_filter_by = 2009

for idx in tqdm(indexes):
    if idx == 0:
        url="https://www.dba.dk/biler/biler/maerke-peugeot/"
    else:
        url = "https://www.dba.dk/biler/biler/maerke-peugeot/side-{}/".format(idx)
        
    scraped_total_cars += scrape_for_cars(url, search_year_flag, year_to_filter_by)
    
print(len(scraped_total_cars))

 20%|██        | 1/5 [00:00<00:02,  1.56it/s]

41
[('https://www.dba.dk/peugeot-207-14-hdi-70/id-1077684148/', '2009', '203000', '18500'), ('https://www.dba.dk/peugeot-206-14-hdi/id-1077071356/', '2009', '215000', '14900'), ('https://www.dba.dk/peugeot-308-diesel-2009-km/id-1073536154/', '2009', '147500', '39500')]


 40%|████      | 2/5 [00:01<00:01,  1.88it/s]

21
[]


 60%|██████    | 3/5 [00:01<00:01,  1.85it/s]

21
[('https://www.dba.dk/peugeot-207-16-hdi-90-diesel/id-505577819/', '2009', '126000', '29900'), ('https://www.dba.dk/peugeot-207-16-cc-benzin/id-505567226/', '2009', '118000', '89900'), ('https://www.dba.dk/peugeot-308-20-hdi-136/id-505489630/', '2009', '230000', '39900')]


 80%|████████  | 4/5 [00:02<00:00,  1.88it/s]

21
[('https://www.dba.dk/peugeot-407-18-performance/id-505542160/', '2009', '168000', '29900'), ('https://www.dba.dk/peugeot-207-16-hdi-110/id-1077766110/', '2009', '192000', '19900')]


100%|██████████| 5/5 [00:02<00:00,  1.79it/s]

21
[('https://www.dba.dk/peugeot-308-16-hdi-110/id-505577260/', '2009', '229000', '18999'), ('https://www.dba.dk/peugeot-107-10-comfort/id-505577176/', '2009', '95000', '33800')]
10


## Exercise 2
Brug Regular Expressions til at sortere de fundne biler på år 2009.

Der er implementeret en løsning hvor den leder efter year_to_filter_by hvis search_year_flag er sat til True

## Exercise 3
Brug Selenium til at tilgå de samme punkter som i forrige opgaver, og vælge den dyreste bil.
Brug BeautifulSoup til at printe bilbeskrivelsen ud.

In [100]:
from selenium import webdriver
import bs4
import json
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

url = 'https://www.dba.dk/biler/biler/maerke-peugeot/'
def get_cars_selenium():
    
    browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    browser.get(url)
    browser.implicitly_wait(2)
    button = browser.find_element_by_id('onetrust-accept-btn-handler')
    button.click()
    
    all_cars = []
    
    number_of_pages = browser.find_element_by_css_selector('.pagination > ul:nth-child(1) > li:nth-child(6) > a:nth-child(1)')
    number_of_pages = int(str(number_of_pages.text).strip())
    
    for n in range(1,number_of_pages-90):
        if(n != 1):
            browser.get(url + 'side-' + str(n))
            
        print(browser.current_url)
        soup = bs4.BeautifulSoup(requests.get(browser.current_url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')
        cell_count = soup.select("tr[class^=dbaListing]")
        print(len(cell_count))
        browser.implicitly_wait(2)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        car_single_page = []
        
        for num in range(4, len(cell_count)+5):
            try:
                km = str(browser.find_element_by_css_selector('tr.dbaListing:nth-child(' + str(num) +') > td:nth-child(3)').text)
                km = km.replace(".","")
                year = str(browser.find_element_by_css_selector('tr.dbaListing:nth-child(' + str(num) +') > td:nth-child(4)').text)
                price = str(browser.find_element_by_css_selector('tr.dbaListing:nth-child(' + str(num) +') > td:nth-child(6) > a:nth-child(1)').text)
                price = price.replace(".","")
                car_single_page.append((year, km, price))
            except:
                pass
        #print(car_single_page)
        all_cars += car_single_page
    return all_cars

total = get_cars_selenium()
print(len(total))
print(total)

[WDM] - ====== WebDriver manager ======
[WDM] - Driver [C:\Users\Gustav\.wdm\drivers\geckodriver\win64\v0.29.0\geckodriver.exe] found in cache




https://www.dba.dk/biler/biler/maerke-peugeot/
41
https://www.dba.dk/biler/biler/maerke-peugeot/side-2/
21
https://www.dba.dk/biler/biler/maerke-peugeot/side-3/
21
https://www.dba.dk/biler/biler/maerke-peugeot/side-4/
21
https://www.dba.dk/biler/biler/maerke-peugeot/side-5/
21
https://www.dba.dk/biler/biler/maerke-peugeot/side-6/
21
146
[('2004', '207000', '12000 kr'), ('2011', '290000', '14900 kr'), ('2008', '308000', '8500 kr'), ('2012', '117000', '55000 kr'), ('2008', '285000', '14999 kr'), ('2007', '251000', '22195 kr'), ('2009', '203000', '18500 kr'), ('2007', '285000', '17900 kr'), ('2010', '195000', '28500 kr'), ('2007', '270000', '25000 kr'), ('2010', '120000', '26999 kr'), ('2001', '275000', '7900 kr'), ('2009', '215000', '14900 kr'), ('2017', '98000', '61990 kr'), ('2010', '229000', '21000 kr'), ('2013', '147000', '29000 kr'), ('2007', '145500', '24900 kr'), ('2012', '147000', '28000 kr'), ('2006', '206000', '22500 kr'), ('2009', '147500', '39500 kr'), ('2004', '268500', '5